# Practical 2: Supply and use tables

## Learning objectives
- Calculate product outputs in different valuations
- Understand the structure of supply tables in practice
- Distinguish valuations and types of productions

In [1]:
# import modules
import pandas as pd

# Import data
supply_ = pd.read_excel("data/modified_supply_basic_price_AT_OECD_2015.xlsx", engine="openpyxl", skipfooter=3)

In [3]:
# organize you row and column labels
supply_col_lb = supply_.iloc[9,7:].to_list()
supply_row_lb = supply_.iloc[11:, 2].to_list()

In [4]:
# isolate array with only values
supply = supply_.iloc[11:, 7:]

In [5]:
# assign clean labels to columns and row
supply.columns = supply_col_lb
supply.index = supply_row_lb

## Exercise 1

1. Look at the product list and select two products

In [6]:
# Print list of products (i.e., index of your table)
print(supply.index)

Index(['Products of agriculture, hunting and related services',
       'Products of forestry, logging and related services',
       'Fish & other fishing products, aquaculture prod., support serv. to fishing',
       'Mining and quarrying', 'Food, beverages and tobacco products',
       'Textiles, wearing apparel, leather and related products',
       'Wood & prod. of wood & cork, exc. furniture, of straw & plaiting materials',
       'Paper and paper products', 'Printing and recording services',
       'Coke and refined petroleum products',
       'Chemicals and chemical products',
       'Basic pharmaceutical products and pharmaceutical preparations',
       'Rubber and plastic products', 'Other non-metallic mineral products',
       'Basic metals',
       'Fabricated metal products, except machinery and equipment',
       'Computer, electronic and optical products', 'Electrical equipment',
       'Machinery and equipment n.e.c.',
       'Motor vehicles, trailers and semi-trailers',


In [7]:
# Select two products
products = ["Electrical equipment", "Chemicals and chemical products"]

2. Can you quantify their total supply, import, and which industries supply them?

In [8]:
# Slice your data
p_1_supply = supply.loc[products[0],:]
p_2_supply = supply.loc[products[1],:]


In [9]:
# Isolate non zero values and drop NaN values
p_1_supply = p_1_supply[p_1_supply != 0].dropna()
p_2_supply = p_2_supply[p_2_supply != 0].dropna()

In [10]:
# Print results
print(f"{products[0]}:\n\n{p_1_supply}\n\n{products[1]}:\n\n{p_2_supply}")

Electrical equipment:

Printing and reproduction of recorded media                                   0.47
Manufacture of chemicals and chemical products                               3.163
Manufacture of rubber and plastics products                                  9.633
Manufacture of other non-metallic mineral products                           0.643
Manufacture of basic metals                                                 29.742
Manufacture of fabricated metal products, except machinery & equipment       17.21
Manufacture of computer, electronic and optical products                    90.671
Manufacture of electrical equipment                                       7208.328
Manufacture of machinery and equipment n.e.c.                              122.559
Manufacture of motor vehicles, trailers and semi-trailers                  249.244
Manufacture of other transport equipment                                    28.114
Repair and installation of machinery and equipment              

3. What is the total product output at basic price and at purchaser price?

    N.b. in the Supply and Use table Non-Deductible VAT is included in the Taxes less subsidies on products (see also page 86 Eurostat Manual of Supply, Use and Input-Output Tables)

Tip: in the file "modified_supply_basic_price_AT_OECD_2015.xlsx" you can check whether the calculations are right in the columns "Total supply at purchasers' prices" and "Total supply at basic prices"

In [11]:
# Total supply at basic and purchaser prices for product 1
p_1_basic_price = p_1_supply.loc[:"Imports, cif"].sum()

p_1_taxes_less_subsidies = p_1_supply.loc["Taxes less subsidies on products"]
p_1_trade_n_transport_margins = p_1_supply.loc["Trade and transport margins"]

p_1_purchaser_price = p_1_basic_price + p_1_taxes_less_subsidies + p_1_trade_n_transport_margins


In [12]:
# Total supply at basic and purchaser prices for product 2
p_2_basic_price = p_2_supply.loc[:"Imports, cif"].sum()

p_2_taxes_less_subsidies = p_2_supply.loc["Taxes less subsidies on products"]
p_2_trade_n_transport_margins = p_2_supply.loc["Trade and transport margins"]

p_2_purchaser_price = (
    p_2_basic_price + p_2_taxes_less_subsidies + p_2_trade_n_transport_margins
)

In [13]:
# basic and purchase prices for product 1
print(
    f"{products[0]}: \n\tBasic price: {p_1_basic_price}\n\tPurchaser price: {p_1_purchaser_price}"
)

# basic and purchase prices for product 2
print(
    f"{products[1]}: \n\tBasic price: {p_2_basic_price}\n\tPurchaser price: {p_2_purchaser_price}"
)

Electrical equipment: 
	Basic price: 15592.110999999999
	Purchaser price: 18259.775
Chemicals and chemical products: 
	Basic price: 26349.754
	Purchaser price: 30646.404


## Exercise 2
1. Look at the list of industries (i.e., columns of the supply table) and select two industries

In [8]:
# Print list of industries (i.e., columns of your table)
print(supply.columns)

Index(['Crop and animal production, hunting and related service activities',
       'Forestry and logging', 'Fishing and aquaculture',
       'Mining and quarrying',
       'Manufacture of food products, beverages and tobacco products',
       'Manufacture of textiles, wearing apparel, leather and related products',
       'Manufacture of wood and of products of wood and cork,etc.',
       'Manufacture of paper and paper products',
       'Printing and reproduction of recorded media',
       'Manufacture of coke and refined petroleum products',
       'Manufacture of chemicals and chemical products',
       'Manufacture of basic pharmaceutical products and preparations',
       'Manufacture of rubber and plastics products',
       'Manufacture of other non-metallic mineral products',
       'Manufacture of basic metals',
       'Manufacture of fabricated metal products, except machinery & equipment',
       'Manufacture of computer, electronic and optical products',
       'Manufacture

In [15]:
# Select two industries
industries = ['Manufacture of coke and refined petroleum products', 'Repair and installation of machinery and equipment']

2. Can you identify the main products and secondary products? 

In [16]:
# Slice your data
i_1_supply = supply.loc[:, industries[0]]
i_2_supply = supply.loc[:, industries[1]]

In [17]:
# Isolate non zero values and sort them from biggest to smallest
i_1_supply = i_1_supply[i_1_supply != 0].sort_values(ascending=False)
i_2_supply = i_2_supply[i_2_supply != 0].sort_values(ascending=False)

# Print results
print(f"{industries[0]}:\n\n{i_1_supply}\n\n{industries[1]}:\n\n{i_2_supply}")

Manufacture of coke and refined petroleum products:

Coke and refined petroleum products                                          3250.968
Chemicals and chemical products                                               801.137
Legal, accounting, head offices services, management consultancy serv.        109.559
Electricity, gas, steam and air conditioning                                    49.28
Computer programming, consultancy and related serv., Information serv.         34.756
Real estate services excluding imputed rents                                   25.273
Warehousing and support services for transportation                            25.065
Wholesale trade services, except of motor vehicles and motorcycles             16.605
Advertising and market research services                                        9.104
Scientific research and development services                                    6.663
Repair and installation services of machinery and equipment                     2.937
C